### Package and Data Imports

In [ ]:
!pip install scanpy python-igraph leidenalg catboost


In [ ]:
import scanpy as sc
from scipy.stats import spearmanr
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
import scipy as sp
import torch
import numpy as np
import pandas as pd
import random
import joblib
# Various Random Forests
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier
from lightgbm.sklearn import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from xgboost import XGBRFClassifier
# For NMF Topic Modelling
from sklearn.metrics import silhouette_score
from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import normalize

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
%cd /content/drive/MyDrive/445_project_data/

#### Setup Data



In [ ]:
adata_nonsmkr = sc.read_text('internal_nonsmokerslung.expression.txt', delimiter='\t').T
adata_smkr = sc.read_text('internal_smokerslung.expression.txt', delimiter='\t').T

df_nonsmkr_meta = pd.read_csv('internal_nonsmokerslung.meta.txt', sep="\t", index_col=0)
df_smkr_meta = pd.read_csv('internal_smokerslung.meta.txt', sep="\t", index_col=0)

In [ ]:
df_smkr_data_and_meta = pd.concat([adata_smkr.to_df(), df_smkr_meta], axis=1)
df_nonsmkr_data_and_meta = pd.concat([adata_nonsmkr.to_df(), df_nonsmkr_meta], axis=1)

In [ ]:
df_nonsmkr_data_and_meta

In [ ]:
adata_nonsmkr.var_names

### Calculate Cell Amounts

In [ ]:
smkr_by_type = {}
nonsmkr_by_type = {}
cellTypes = df_smkr_data_and_meta['CellType'].unique()
for cellType in cellTypes:
    smkr_by_type[cellType] = df_smkr_data_and_meta[df_smkr_data_and_meta['CellType'] == cellType].copy()
    nonsmkr_by_type[cellType] = df_nonsmkr_data_and_meta[df_nonsmkr_data_and_meta['CellType'] == cellType].copy()

cellType_smkr_sizes = [cellgroup.shape[0] for cellgroup in smkr_by_type.values()]
cellType_nonsmkr_sizes = [cellgroup.shape[0] for cellgroup in nonsmkr_by_type.values()]

In [ ]:
bar_width = 0.35
even_spacing = np.arange(len(cellType_smkr_sizes))
fig, ax = plt.subplots()
bars1 = ax.bar(even_spacing - bar_width/2, cellType_smkr_sizes, bar_width, label='non-smoker')
bars2 = ax.bar(even_spacing + bar_width/2, cellType_nonsmkr_sizes, bar_width, label='smoker')

ax.set_ylabel('Number of Cells')
ax.set_title('Numbers of Cell Types in Smokers and Non-smokers')
ax.set_xticks(even_spacing)
ax.set_xticklabels(smkr_by_type.keys())
ax.legend()

plt.xticks(rotation=90)
plt.show()

### Calculate Correlation Matrix

In [ ]:
X = adata_nonsmkr.X
# corr_matrix_nonsmkr = np.corrcoef(X, rowvar=False) #pearson correlation
corr_matrix_nonsmkr, spearman_p_vals = spearmanr(X) #spearman correlation
X = adata_smkr.X
# corr_matrix_smkr = np.corrcoef(X, rowvar=False) #pearson correlation
corr_matrix_smkr, spearman_p_vals = spearmanr(X) #spearman correlation

corr_matrix_smkr

In [ ]:
fig, axises = plt.subplots(ncols=2, figsize=(10, 4))
sns.heatmap(corr_matrix_nonsmkr, cmap='coolwarm', vmin=-1, vmax=1, ax=axises[0],
            xticklabels=adata_nonsmkr.var_names, yticklabels=adata_nonsmkr.var_names)
axises[0].set_title("Non-smokers")
sns.heatmap(corr_matrix_smkr, cmap='coolwarm', vmin=-1, vmax=1, ax=axises[1],
            xticklabels=adata_smkr.var_names, yticklabels=adata_smkr.var_names)
axises[1].set_title("Smokers")
plt.suptitle("Spearman correlation for gene pairs in COVID19 patients", fontsize=12, y=1.0)
plt.show()

### Calculate Mutual Information Matrix

In [ ]:
X = adata_nonsmkr.X
def make_mutual_info_matrix(X_):
    num_feat = X_.shape[1]
    toStack = [[0 for i in range(num_feat)] for j in range(num_feat)]
    for col in range(num_feat):
        toStack[col] = mutual_info_classif(X_, X_[:,col], n_neighbors=3)
    return np.stack(toStack, axis=1)

In [ ]:
mi_matrix_nonsmkr = make_mutual_info_matrix(adata_nonsmkr.X)
mi_matrix_smkr = make_mutual_info_matrix(adata_smkr.X)

In [ ]:
fig, axises = plt.subplots(ncols=2, figsize=(10, 4))
sns.heatmap(mi_matrix_nonsmkr, cmap='BuPu', vmax=0.2, ax=axises[0],
            xticklabels=adata_smkr.var_names, yticklabels=adata_smkr.var_names)
axises[0].set_title("Non-smokers")
sns.heatmap(mi_matrix_smkr, cmap='BuPu', vmax=0.2, ax=axises[1],
            xticklabels=adata_smkr.var_names, yticklabels=adata_smkr.var_names)
axises[1].set_title("Smokers")
plt.suptitle("Mutual Information for gene pairs in COVID19 patients", fontsize=12, y=1.0)
plt.show()

### Random Forest Classifiers

#### Create features X and target y



###### make X and y without meta data



In [ ]:
# pd.DataFrame(adata_smkr.X)
# sp.sparse.csr_matrix(adata_smkr.X)
X_no_meta = np.concatenate([adata_nonsmkr.X, adata_smkr.X]) #combine ndarrays

In [ ]:
print(f'# of non-smoker examples is {adata_nonsmkr.X.shape[0]} \n',
      f'# of smoker examples is {adata_smkr.X.shape[0]} \n',
      f'ratio is {adata_smkr.X.shape[0] / adata_nonsmkr.X.shape[0]}')

In [ ]:
def make_target_from_two_datasets(matrix1: np.ndarray, matrix2: np.ndarray):
    y_1 = [False]*(adata_smkr.X.shape[0])
    y_2 = [True]*(adata_nonsmkr.X.shape[0])
    return np.concatenate([y_1, y_2])
y = make_target_from_two_datasets(adata_nonsmkr.X, adata_smkr)

In [ ]:
# shuffle data and split (keeping target class proportions the same)
X_train_nm, X_test_nm, y_train_nm, y_test_nm = train_test_split(X_no_meta, y, test_size=0.3, shuffle=True, stratify=y)
# over-sample from the least frequent target class to balance the classes
randomOverSampler = RandomOverSampler()
X_train_nm, y_train_nm = randomOverSampler.fit_resample(X_train_nm, y_train_nm)
print(f'# of non-smoker training examples is {y_train_nm[y_train_nm==False].shape[0]} \n',
      f'# of smoker training examples is {y_train_nm[y_train_nm==True].shape[0]}')

###### make X and y with meta data



In [ ]:
X_meta = pd.concat([df_nonsmkr_data_and_meta, df_smkr_data_and_meta])

In [ ]:
# def make_target_from_two_datasets(df1: pd.DataFrame, df2: pd.DataFrame):
#     y_1 = [False] * (df1.shape[0])
#     y_2 = [True] * (df2.shape[0])
#     return np.concatenate([y_1, y_2])
# y = make_target_from_two_datasets(df_nonsmkr_data_and_meta, df_smkr_data_and_meta)


In [ ]:
# shuffle data and split (keeping target class proportions the same)
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(X_meta, y, test_size=0.3, shuffle=True, stratify=y)
# over-sample from the least frequent target class to balance the classes
randomOverSampler = RandomOverSampler()
X_train_m, y_train_m = randomOverSampler.fit_resample(X_train_m, y_train_m)
print(f'# of non-smoker training examples is {y_train_m[y_train_m == False].shape[0]} \n',
      f'# of smoker training examples is {y_train_m[y_train_m == True].shape[0]}')

#### Single Random Forest

###### cross validation

In [ ]:
rf_model = XGBRFClassifier(tree_method='gpu_hist')
# rf_model = RandomForestClassifier()
rf_grid_search = GridSearchCV(rf_model, verbose=3, n_jobs=-4, return_train_score=True, param_grid={
    'max_depth':    [1, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44],
    'n_estimators': [10]})

In [ ]:
rf_grid_search.fit(X_train_nm, y_train_nm)
print(rf_grid_search.score(X_train_nm, y_train_nm))

In [ ]:
results = rf_grid_search.cv_results_
#pd.DataFrame(results)

In [ ]:
def plot_cross_validation(param_list: list[int], mean_train_score: list[float], mean_validation_score: list[float]):
    fig, ax = plt.subplots()
    ax.plot(param_list, mean_train_score, "lightblue")
    ax.plot(param_list, mean_train_score, 'bo', label="training_score")
    ax.plot(param_list, mean_validation_score, "pink")
    ax.plot(param_list, mean_validation_score, 'ro', label="validation_score")
    ax.set_title('Random Forest Cross Validation Grid Search Results')
    ax.set_xlabel('Max Tree Depth')
    ax.set_ylabel('Mean Validation Score')
    fig.set_size_inches(8, 3)
    plt.ylim(0.5, 0.8)
    plt.grid(axis = 'y')
    ax.set_xticks(param_list)
    ax.legend()
    # plt.xlim(param_list[0], param_list[-1])
    plt.show()

In [ ]:
plot_cross_validation(results['param_max_depth'].tolist(), results['mean_train_score'].tolist(), results['mean_test_score'].tolist())

###### try lgbm, xgboost, catboost

In [ ]:
lgbmModel = LGBMClassifier()
catBoostModel = CatBoostClassifier(verbose=0)
xgbModel = XGBClassifier(eval_metric="logloss", nthread=-1, verbosity=1)

tree_classifiers = {
    "random forest": rf_model,
    "LightGBM": lgbmModel,
    "CatBoost": catBoostModel,
    "XGBoost": xgbModel,
}

In [ ]:
# Hyperparameter Optimization
lgbm_grid_search = GridSearchCV(lgbmModel, n_jobs=-1, return_train_score=True, param_grid={
    'learning_rate': [0.1],
    'n_estimators': [100]
})
catboost_grid_search = GridSearchCV(catBoostModel, n_jobs=-1, return_train_score=True, param_grid={
    'learning_rate':    [1],
    'n_estimators': [1000]
})
xgboost_grid_search = GridSearchCV(xgbModel, n_jobs=-1, return_train_score=True, param_grid={
    'learning_rate':    [1],
    'n_estimators': [100],
    'nthread':[-1]
})

In [ ]:
lgbm_grid_search.fit(X_train_nm, y_train_nm)
print(lgbm_grid_search.score(X_test_nm, y_test_nm))

In [ ]:
y_test_string = np.array([str(target) for target in y_test_nm])
catboost_grid_search.fit(X_train_nm, y_train_nm)
print(catboost_grid_search.score(X_test_nm, y_test_string))

In [ ]:
xgboost_grid_search.fit(X_train_nm, y_train_nm)
print(xgboost_grid_search.score(X_test_nm, y_test_nm))

###### fit / score

In [ ]:
SINGLE_RF_BEST_DEPTH = 32
single_rf_model =  RandomForestClassifier(max_depth=SINGLE_RF_BEST_DEPTH, n_estimators=100)
single_rf_model.fit(X_train_nm, y_train_nm)
single_rf_model.score(X_test_nm, y_test_nm)

In [ ]:
data = {"Importance": single_rf_model.feature_importances_}
#pd.DataFrame(data=data, index=adata_smkr.var_names).sort_values(by="Importance", ascending=False)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.bar(adata_smkr.var_names, single_rf_model.feature_importances_)
#ax.set_title("Feature Importances in Random Forest")
ax.set_ylabel("Feature Importance")
plt.xticks(rotation=90)
fig.set_size_inches(8, 3)
plt.show()

#### Random Forests per Cell Type (Hard)

##### class

In [ ]:
class RandomForestByCellType:
    def __init__(self, max_depths=None, n_estimators=[1]):
        if max_depths is None:
            self.max_depths = [1,2,3] # 8, 12, 16, 20, 24, 28, 32, 36, 40, 44
        else:
            self.max_depths = max_depths
        self.n_estimators = n_estimators
        self.gridSearches = {}
        self.models = {}
        self.cv_results = {}
        self.graphs = {}
        self.num_cells_by_type_cv = {}
        self.num_cells_by_type_fit = {}

    def set_best_params_from_cv_results(self, cellTypes_):
        self.best_depth = {}
        self.best_n_estimators = {}
        for cellType in cellTypes_:
            cellType_cv_result = self.cv_results[cellType]
            self.best_depth[cellType] = cellType_cv_result.loc[cellType_cv_result["mean_test_score"].idxmax(),"param_max_depth"]
            self.best_n_estimators[cellType] = cellType_cv_result.loc[cellType_cv_result["mean_test_score"].idxmax(),"param_n_estimators"]

    def score(self, X_, y_):
        y_hat = self.predict(X_)
        count = 0
        for i in range(len(y_hat)):
            if y_hat[i] == y_[i]:
                count += 1
        return count / len(y_hat)

In [ ]:
class RandomForestByCellHardCat(RandomForestByCellType):
    def __init__(self, max_depths=None, n_estimators=[1]):
        super().__init__(max_depths=max_depths, n_estimators=n_estimators)

    def subset_cells_by_type(self, X_, y_, cellType):
        X_subset = X_.loc[X_["CellType"] == cellType].iloc[:,0:27]
        y_subset = y_[X_subset.index]
        return (X_subset, y_subset)

    def cross_validate(self, X_, y_, cellTypes_, verbose=True):
        if verbose: cv_verbosity = 3
        else: cv_verbosity = 0
        for cellType in cellTypes_:
            (X_subset, y_subset) = self.subset_cells_by_type(X_, y_, cellType)
            self.num_cells_by_type_cv[cellType] = len(y_subset)
            if verbose:
                print(f'cross validating random tree for {len(y_subset)} {cellType} cells')
            rf_model = RandomForestClassifier()
            #print(f'verbosity={cv_verbosity}')
            self.gridSearches[cellType] = GridSearchCV(rf_model, n_jobs=-1, verbose=cv_verbosity, return_train_score=True, param_grid={
                'max_depth': self.max_depths,
                'n_estimators': self.n_estimators
            })
            self.gridSearches[cellType].fit(X_subset, y_subset)
            self.cv_results[cellType] = pd.DataFrame(self.gridSearches[cellType].cv_results_)

    def fit(self, X_, y_, cellTypes_, verbose=True):
        for cellType in cellTypes_:
            (X_subset, y_subset) = self.subset_cells_by_type(X_, y_, cellType)
            self.num_cells_by_type_fit[cellType] = len(y_subset)
            if verbose:
                print(f'fitting random tree for {len(y_subset)} {cellType} cells')
            self.models[cellType] = RandomForestClassifier(max_depth=self.best_depth[cellType], n_estimators=self.best_n_estimators[cellType])
            self.models[cellType].fit(X_subset, y_subset)

    def predict(self, X_, verbose=True):
        n = X_.shape[0]
        K = len(self.models.keys())
        self._predictions = {}
        self._final_predictions = np.empty(n)
        for cellType, model in self.models.items():
            if verbose:
                print(f'predicting with random tree for {cellType} cells')
            self._predictions[cellType] = model.predict(X_.iloc[:,0:27])
        for i in range(n):
            cellType = X_["CellType"].iloc[i]
            self._final_predictions[i] = self._predictions[cellType][i]
        return self._final_predictions


##### cross validation

In [ ]:
cellTypes = df_smkr_data_and_meta['CellType'].unique()
rf_by_cell_type = RandomForestByCellHardCat(max_depths=[1,5,10,20,30,40], n_estimators=[10])
rf_by_cell_type.cross_validate(X_train_m, y_train_m, cellTypes)

In [ ]:
rf_by_cell_type.cv_results

In [ ]:
rf_by_cell_type.set_best_params_from_cv_results(cellTypes)
best_depth_by_cell_type = []
for cellType in cellTypes:
    best_depth_by_cell_type.append(rf_by_cell_type.best_depth[cellType])
pd.Series(data=best_depth_by_cell_type, index=cellTypes)

In [ ]:
def plot_cross_validation_per_cell(cv_results: dict[str, list[float]], cellTypes: list[str], num_cells: list[int]):
    for i in range(len(cellTypes)):
        dash_lens = [(1,1),(2,3),(5,1)]
        sns.lineplot(data=cv_results[cellTypes[i]], x='param_max_depth', y='mean_test_score',
                     dashes=dash_lens[i%len(dash_lens)], marker="o", label=f'{cellTypes[i]} - ({num_cells[i]} cells)')
    plt.xlabel('max random tree depth')
    plt.ylabel('mean validation score')
    plt.ylim(0.5, 1)
    plt.grid(axis = 'y')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.title("Cross Validation of Random Forest for each Cell Type")

num_cells_per_type = [rf_by_cell_type.num_cells_by_type_cv[cellType] for cellType in cellTypes]
plot_cross_validation_per_cell(rf_by_cell_type.cv_results, cellTypes, num_cells_per_type)

##### fit / score

In [ ]:
rf_by_cell_type.fit(X_train_m, y_train_m, cellTypes)

In [ ]:
rf_by_cell_type.score(X_train_m, y_train_m)

In [ ]:
# grid_search_by_cell_type.score(X_train.iloc[0:10,:], y_train)
rf_by_cell_type.score(X_test_m, y_test_m)

In [ ]:
def make_feature_importance_matrix(model_dict, cellTypes):
    num_feat = 27
    num_cell_types = len(cellTypes)
    toStack = [[0 for i in range(num_feat)] for j in range(num_cell_types)]
    for col in range(num_cell_types):
        toStack[col] = model_dict[cellTypes[col]].feature_importances_
    return np.stack(toStack, axis=1).T
feat_importances_by_cell_type = make_feature_importance_matrix(rf_by_cell_type.models, cellTypes)

In [ ]:
def test_accuracy_and_sizes_by_cell_type(model_, X_, y_, cellTypes_):
    accuracies = []
    sizes = []
    for cellType in cellTypes_:
        (X_subset, y_subset) = model_.subset_cells_by_type(X_.reset_index(drop=True), y_, cellType)
        accuracies.append(model_.models[cellType].score(X_subset, y_subset))
        sizes.append(len(y_subset))
    return np.array([accuracies,sizes])

test_accuracy_and_sizes_by_cell_type = test_accuracy_and_sizes_by_cell_type(rf_by_cell_type, X_test_m, y_test_m, cellTypes)
print(test_accuracy_and_sizes_by_cell_type)

In [ ]:
fig = plt.figure(figsize=(10, 4))
gs = gridspec.GridSpec(1, 3, width_ratios=[5, 1, 1])

# left subplot
ax1 = plt.subplot(gs[0])
sns.heatmap(feat_importances_by_cell_type, cmap='BuPu', vmax=0.2, ax=ax1,
            xticklabels=adata_smkr.var_names, yticklabels=cellTypes)
ax1.set_title("Feature Importances\n(mean decrease in gini importance)")

# mid subplot
ax2 = plt.subplot(gs[1])
sns.heatmap(test_accuracy_and_sizes_by_cell_type[0,:].reshape((15,1))*100, vmin=50, vmax=100, cmap='YlOrBr', ax=ax2,
            xticklabels=["test %\naccuracy"], yticklabels=[], annot=True, fmt=".1f")
plt.xticks(rotation=90)

# right subplot
ax3 = plt.subplot(gs[2])
sns.heatmap(test_accuracy_and_sizes_by_cell_type[1,:].reshape((15,1)), vmin=0, vmax=30000, cmap='Greys', ax=ax3,
            xticklabels=["# of\ntest cells"], yticklabels=[], annot=True, fmt=".0f")
plt.xticks(rotation=90)
plt.show()

In [ ]:
# grid_search_by_cell_type.cv_results['Immune (Myeloid)']

#### Random Forests per Cell Type (Soft)

##### class

In [ ]:
class RandomForestByCellSoftCat(RandomForestByCellType):
    def __init__(self, max_depths=None, n_estimators=[1]):
        super().__init__(max_depths=max_depths, n_estimators=n_estimators)

    def subset_cells_by_topic(self, X_, y_, topic_props_, topic):
        cell_indexes = [i for i in range(len(y_))]
        weights = topic_props_[:,topic]
        total_weight = sum(weights)
        probs = [weight/total_weight for weight in weights]
        chosen_indexes = random.choices(cell_indexes, weights=probs, k=round(total_weight))
        X_subset = X_[chosen_indexes,:]
        y_subset = y_[chosen_indexes]
        return (X_subset, y_subset)

    def cross_validate(self, X_, y_, topic_props_, verbose=True):
        self.sample_sizes = {}
        for cellType in range(topic_props_.shape[1]):
            (X_subset, y_subset) = self.subset_cells_by_topic(X_, y_, topic_props_, cellType)
            self.sample_sizes[cellType] = len(y_subset)
            self.num_cells_by_type_cv[cellType] = len(y_subset)
            if verbose:
                print(f'cross validating random tree for {len(y_subset)} topic #{cellType} cells')
            rf_model = RandomForestClassifier()
            self.gridSearches[cellType] = GridSearchCV(rf_model, n_jobs=-1, return_train_score=True, param_grid={
                'max_depth': self.max_depths,
                'n_estimators': self.n_estimators
            })
            self.gridSearches[cellType].fit(X_subset, y_subset)
            self.cv_results[cellType] = pd.DataFrame(self.gridSearches[cellType].cv_results_)

    def fit(self, X_, y_, topic_props_, verbose=True):
        for cellType in range(topic_props_.shape[1]):
            (X_subset, y_subset) = self.subset_cells_by_topic(X_, y_, topic_props_, cellType)
            self.num_cells_by_type_fit[cellType] = len(y_subset)
            if verbose:
                print(f'fitting random tree for {len(y_subset)} cells in topic #{cellType}')
            self.models[cellType] = RandomForestClassifier(max_depth=self.best_depth[cellType], n_estimators=self.best_n_estimators[cellType])
            self.models[cellType].fit(X_subset, y_subset)

    def find_predictions_by_topic(self, X_, topic_props_):
        n_topics = topic_props_.shape[1]
        n = X_.shape[0]
        toStack = [self.models[i].predict(X_) for i in range(n_topics)]
        self._predictions = np.stack(toStack, axis=1)
    
    def predict_proba(self, X_, topic_props_):
        n = X_.shape[0]
        self.find_predictions_by_topic(X_, topic_props_)
        weighted_predictions = self._predictions*topic_props_
        self._final_predictions_proba = np.array([sum(weighted_predictions[i,:]) for i in range(n)])
        return self._final_predictions_proba

    def predict(self, X_, topic_props_):
        self._final_predictions = np.array([round(val) for val in self.predict_proba(X_, topic_props_)])
        return self._final_predictions

    def score(self, X_, y_, topic_props_):
        y_hat = self.predict(X_, topic_props_)
        count = 0
        for i in range(len(y_hat)):
            if y_hat[i] == y_[i]:
                count += 1
        return count / len(y_hat)

In [ ]:
n_topics = 8
model = NMF(n_components=n_topics, init="random", random_state=0)
W_train_normed = normalize(model.fit_transform(X_train_nm), norm='l1', axis=1)
W_test_normed = normalize(model.fit_transform(X_test_nm), norm='l1', axis=1)
#display(pd.DataFrame(W_train_normed))

##### cross validation

In [ ]:
rf_by_topic = RandomForestByCellSoftCat(max_depths=[1,5,10,20,30,40], n_estimators=[20])
#test_class.subset_cells_by_topic(X_train_nm, y_train_nm, W_train_normed, 2)
rf_by_topic.cross_validate(X_train_nm, y_train_nm, W_train_normed)

In [ ]:
topic_names = [f'topic #{i}' for i in range(n_topics)]
topic_indexes = [i for i in range(n_topics)]
sample_sizes = [rf_by_topic.sample_sizes[i] for i in range(n_topics)]

In [ ]:
def plot_cross_validation_per_topic(cv_results: dict[str, list[float]], n_topics: int, num_cells: list[int]):
    for i in range(n_topics):
        dash_lens = [(1,1),(2,3),(5,1)]
        sns.lineplot(data=cv_results[i], x='param_max_depth', y='mean_test_score',
                     dashes=dash_lens[i%len(dash_lens)], marker="o", label=f'topic #{i} - ({num_cells[i]} cells)')
    plt.xlabel('max random tree depth')
    plt.ylabel('mean validation score')
    plt.ylim(0.5, 1)
    plt.grid(axis = 'y')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.title("Cross Validation of Random Forest for each Cell Type")
plot_cross_validation_per_topic(rf_by_topic.cv_results, n_topics, sample_sizes)

In [ ]:
rf_by_topic.set_best_params_from_cv_results(topic_indexes)
rf_by_topic.best_depth

##### fit / score

In [ ]:
rf_by_topic.fit(X_train_nm, y_train_nm, W_train_normed)

In [ ]:
print(rf_by_topic.predict_proba(X_train_nm, W_train_normed))

In [ ]:
print(rf_by_topic.score(X_train_nm, y_train_nm, W_train_normed))

In [ ]:
print(rf_by_topic.score(X_test_nm, y_test_nm, W_test_normed))

In [ ]:
def make_feature_importance_matrix(model_dict, cellTypes):
    num_feat = 27
    num_cell_types = len(cellTypes)
    toStack = [[0 for i in range(num_feat)] for j in range(num_cell_types)]
    for col in range(num_cell_types):
        toStack[col] = model_dict[cellTypes[col]].feature_importances_
    return np.stack(toStack, axis=1).T
feat_importances_by_cell_type = make_feature_importance_matrix(rf_by_topic.models, topic_indexes)

In [ ]:
def make_test_accuracy_and_sizes_array(model_, X_, y_, topic_props_, cellTypes_):
    accuracies = []
    sizes = []
    for cellType in cellTypes_:
        (X_subset, y_subset) = model_.subset_cells_by_topic(X_, y_, topic_props_, cellType)
        accuracies.append(model_.models[cellType].score(X_subset, y_subset))
        sizes.append(len(y_subset))
    return np.array([accuracies,sizes])
test_accuracy_and_sizes_by_cell_type = make_test_accuracy_and_sizes_array(rf_by_topic, X_test_nm, y_test_nm, W_test_normed, topic_indexes)
print(test_accuracy_and_sizes_by_cell_type)

In [ ]:
fig = plt.figure(figsize=(10, 3))
gs = gridspec.GridSpec(1, 3, width_ratios=[5, 1, 1])

# left subplot
ax1 = plt.subplot(gs[0])
sns.heatmap(feat_importances_by_cell_type, cmap='BuPu', vmax=0.2, ax=ax1,
            xticklabels=adata_smkr.var_names, yticklabels=topic_names)
ax1.set_title("Feature Importances\n(mean decrease in gini importance)")

# mid subplot
ax2 = plt.subplot(gs[1])
sns.heatmap(test_accuracy_and_sizes_by_cell_type[0,:].reshape((8,1))*100, vmin=40, vmax=100, cmap='YlOrBr', ax=ax2,
            xticklabels=["test %\naccuracy"], yticklabels=[], annot=True, fmt=".1f")
plt.xticks(rotation=90)

# right subplot
ax3 = plt.subplot(gs[2])
sns.heatmap(test_accuracy_and_sizes_by_cell_type[1,:].reshape((8,1)), vmin=0, vmax=30000, cmap='Greys', ax=ax3,
            xticklabels=["# of\ntest cells"], yticklabels=[], annot=True, fmt=".0f")
plt.xticks(rotation=90)
plt.show()

# Garbage / Experimental

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn.base")
rf_test = grid_search_by_cell_type.models[Xabc.iloc[0,:]["CellType"]]

rf_test.predict(Xabc.iloc[4,0:27].values.reshape(1,-1))